In [118]:
from braket.tracking import Tracker
t = Tracker().start()

# IMPORTS and SETUP

In [119]:
# general imports
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import time
from datetime import datetime
# magic word for producing visualizations in notebook
%matplotlib inline

In [120]:
# Ensure consistent results
np.random.seed(0)

# Flag to trigger writing results plot to file
SAVE_FIG = False

In [121]:
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, AngledGate, QubitSet
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsSession

In [122]:
# Set up device: Local Simulator
device = LocalSimulator()

## PROBLEM SETUP

In [123]:
# helper function to set up interaction term
def get_ising_model(n_qubits):
    """
    function to setup Ising interaction term
    """
    # set number of qubits
    ising = np.zeros((pow(2,n_qubits), pow(2,n_qubits)), dtype=int, order='C')
    # set nearest-neighbour interactions to nonzero values only

    ising[0][0] = 4
    ising[3][3] = 4
    print('Ising matrix:\n', ising)
    
    return ising

In [124]:
# function to build the VQE ansatz
def circuit(params, n_qubits):
    """
    function to return full VQE circuit ansatz
    input: parameter list with three parameters 
    """
    
    # instantiate circuit object
    circuit = Circuit()
    
    #l_bfgs anzats, linear
    for i in range(3):
        for qubit_index in range(n_qubits):
            RY_theta_index = i*2*n_qubits + qubit_index
            RZ_theta_index = RY_theta_index + n_qubits
            circuit.rz(qubit_index, params[RY_theta_index]).ry(qubit_index, params[RZ_theta_index])
            
        if(i != 2):
            circuit.cnot(control=0, target=1)
            
    return circuit


# function that computes cost function for given params
def objective_function(params, b_field, n_qubits, n_shots, verbose=False):
    """
    objective function takes a list of variational parameters as input,
    and returns the cost associated with those parameters
    """
    
    global CYCLE
    CYCLE += 1
    
    if verbose:
        print('==================================' * 2)
        print('Calling the quantum circuit. Cycle:', CYCLE)
    
    # obtain a quantum circuit instance from the parameters
    vqe_circuit = circuit(params, n_qubits)
    
    # Computations are independent of one another, so can be triggered in parallel
    # run the circuit on appropriate device
    if isinstance(device, LocalSimulator):
        task_zz = device.run(vqe_circuit, shots=n_shots)
    else: 
        task_zz = device.run(vqe_circuit, shots=n_shots)

    # Hb term: construct the circuit for measuring in the X-basis
    H_on_all = Circuit().h(range(0, n_qubits))
    vqe_circuit.add(H_on_all)
    
    # run the circuit (with H rotation at end)
    if isinstance(device, LocalSimulator):
        task_b = device.run(vqe_circuit, shots=n_shots)
    else:
        task_b = device.run(vqe_circuit, shots=n_shots)

    # Collect results from devices (wait for results, if necessary)
    result_zz = task_zz.result()
    result_b = task_b.result()
    
    # Compute Hzz term
    # convert results (0 and 1) to ising (1 and -1)
    meas_ising = result_zz.measurements
    meas_ising[meas_ising == 1] = -1
    meas_ising[meas_ising == 0] = 1
    
    # Hzz term: get all energies (for every shot): (n_shots, 1) vector
    all_energies_zz = np.diag(np.dot(meas_ising, np.dot(J, np.transpose(meas_ising))))
    
    # Hzz term: get approx energy expectation value (factor 1/4 for Pauli vs spin 1/2) 
    energy_expect_zz = 0.25*np.sum(all_energies_zz)/n_shots
    
    # Compute Hb term
    # convert results (0 and 1) to ising (1 and -1)
    meas_ising = result_b.measurements
    meas_ising[meas_ising == 1] = -1
    meas_ising[meas_ising == 0] = 1
    
    # Hb term: get all energies (for every shot): (n_shots, 1) vector
    # factor 1/2 for Pauli vs spin 1/2
    energy_expect_b = -1*b_field/2*meas_ising.sum()/n_shots
    
    # get total energy expectation value
    energy_expect = energy_expect_zz + energy_expect_b
    # per site
    energy_expect_ind = energy_expect/n_qubits
    
    # print energy expectation value
    if verbose:
        print('Energy expectation value:', energy_expect)
        print('Energy expectation value (per particle):', energy_expect_ind)

    return energy_expect


# The function to execute the training: run classical minimization.
def train(b_field, options, n_qubits, n_shots, n_initial=10, verbose=False):
    """
    function to run VQE algorithm with several random seeds for initialization
    """
    print('Starting the training.')
    
    if verbose:
        print('==================================' * 3)
        print('Running VQE OPTIMIZATION.')
    
    # initialize vectors for results per random seed
    cost_energy = []
    angles = []
    
    # optimize for different random initializations: avoid local optima
    for ii in range(n_initial):
        
        #print counter
        if verbose:
            run_init = ii+1
            print('Running VQE OPTIMIZATION for random seed NUMBER', run_init)
        
        # randomly initialize variational parameters within appropriate bounds
        params0 = np.random.uniform(0, 2 * np.pi, 32).tolist()
        # set bounds for search space
        bnds = [(0, 2 * np.pi) for _ in range(int(len(params0)))]

        # run classical optimization
        result = minimize(objective_function, params0, args=(b_field, n_qubits, n_shots, verbose), 
                          options=options, method='SLSQP', bounds=bnds)

        # store result of classical optimization
        result_energy = result.fun
        cost_energy.append(result_energy)
        result_angle = result.x
        angles.append(result_angle)
        if verbose:
            print('Optimal avg energy:', result_energy)
            print('Optimal angles:', result_angle)
        
        # reset cycle count 
        global CYCLE
        CYCLE = 0
    
    # store energy minimum (over different initial configurations)
    energy_min = np.min(cost_energy)
    optim_angles = angles[np.argmin(cost_energy)]
    if verbose:
        print('Energy per initial seeds:', cost_energy)
        print('Minimal energy:', energy_min)
        print('Optimal variational angles:', optim_angles)
    
    return energy_min

## Illustration of the VQE ansatz

In [125]:
# visualize VQE circuit example
N = 2 
initial_thetas = np.random.uniform(low=0, high=360, size=32)
vqe_circuit = circuit(initial_thetas, N)

print('1. Printing VQE test circuit:\n')
print(vqe_circuit)



1. Printing VQE test circuit:

T  : |    0     |    1     |2|    3     |    4     |5|    6     |    7     |
                                                                            
q0 : -Rz(197.57)-Ry(216.99)-C-Rz(152.52)-Ry(157.53)-C-Rz(346.92)-Ry(285.02)-
                            |                       |                       
q1 : -Rz(257.47)-Ry(196.16)-X-Rz(232.52)-Ry(321.04)-X-Rz(138.04)-Ry(190.40)-

T  : |    0     |    1     |2|    3     |    4     |5|    6     |    7     |


## VQE SIMULATION ON LOCAL SIMULATOR

In [126]:
# set up the problem
SHOTS = 4
N = 2 # number of qubits
n_initial = 30 # number of random seeds to explore optimization landscape
verbose = False # control amount of print output

# set counters
CYCLE = 0

# set up Ising matrix with nearest neighbour interactions and PBC
J = get_ising_interactions(N)

# set options for classical optimization
if verbose:
    options = {'disp': True}
else: 
    options = {}

# kick off training
start = time.time()

# parameter scan
stepsize = 0.5
xvalues = np.arange(0, 2+stepsize, stepsize)
results = []
results_site = []

for bb in xvalues:
    b_field = bb
    energy_min = train(b_field, options=options, n_qubits=N, n_shots=SHOTS, n_initial=n_initial, 
                verbose=verbose)
    results.append(energy_min)
    results_site.append(energy_min/N)
    
    # reset counters
    CYCLE = 0

end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)

# print optimized results
print('Optimal energies:', results)

Ising matrix:
 [[4 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 4]]
Starting the training.


ValueError: shapes (4,4) and (2,4) not aligned: 4 (dim 1) != 2 (dim 0)